In [2]:
import numpy as np
import pandas as pd
import re, sys, os, csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import string    

In [18]:
def prep(tweet):
    #pattern = re.compile(r"(.)\1{2,}")
    #tweet = pattern.sub(r"\1\1", tweet)
    #tweet = re.sub(r'http.?://[^\s]+[\s]?', '', tweet)
    tweet = re.sub('\d+', '', tweet)
    #punct = string.punctuation
    #trantab = str.maketrans(punct, len(punct) * ' ')  # Every punctuation symbol will be replaced by a space
    #tweet = tweet.translate(trantab)
    #tweet = tweet.lower()
    
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)     
        
    ps = PorterStemmer()
    words = tweet.split()
    stemmed_words = [ps.stem(word) for word in words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in stemmed_words]
    tweet = " ".join(lemma_words)
    return tweet

In [19]:
def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open('dataset/wordlists/merged.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def fix_hashtag(text):
    text = text.group().split(":")[0]
    text = text[1:] # remove '#'
    try:
        test = int(text[0])
        text = text[1:]
    except:
        pass
    output = ' '.join(viterbi_segment(text)[0])
    #print(output)
    return output

def clean_tweet( tweet):
        tweet = tweet.lower()
        tweet = re.sub("(#[A-Za-z0-9]+)", fix_hashtag, tweet)
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "empty":return 0
    elif label == "sadness":return 2
    elif label == "enthusiasm":return 1
    elif label == "neutral":return 0
    elif label == "worry":return 2
    elif label == "surprise":return 1
    elif label == "love":return 3
    elif label == "fun":return 1
    elif label == "hate":return 4
    elif label == "happiness":return 1
    elif label == "boredom":return 0
    elif label == "relief":return 1
    elif label == "anger":return 4
    

In [20]:
def read_csv(file, lst):
    with open(file, newline='', encoding="utf8") as f:
        reader = csv.reader(f)
        for i,row in enumerate(reader):
            if i > 0:
                tweet = clean_tweet(str(row[1]))
                lst.append(tweet)

In [24]:
#hate = []
#read_csv("hate.csv", hate)

#worry = []
#read_csv("raw_data/worried.csv", worry)
#read_csv("worry.csv", worry)
#read_csv("raw_data/anxiety.csv", worry)
#happy = []
#read_csv("raw_data/happy.csv", happy)
anger = []
read_csv("crawled_csv/angry.csv", anger)

dataWriter = csv.writer(open('crawled_csv/anger_processes.csv', 'w'), delimiter=',',lineterminator="\n")
for tweet in anger:
    dataWriter.writerow([tweet, 4])
      
print("Done!")

Done!


In [6]:
count = 0
with open('crawled_csv/anger_processes.csv', encoding = "utf8") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count+=1
print(count)

757


# Shuffle csv files

In [ ]:
import random
fid = open("csv/emotion_data5.csv", "r")
li = fid.readlines()
fid.close()
print(li)

random.shuffle(li)
print(li)

fid = open("csv/emotion_data4.csv", "w")
fid.writelines(li)
fid.close()